In [68]:
import torch
import wandb
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from torchvision.transforms import RandomErasing

In [69]:
# Check if CUDA is available
if torch.cuda.is_available():
    # Set the default tensor type to CUDA tensors
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

    # Define the device as the first visible cuda device if available
    device = torch.device("cuda")
    print(f"Running on {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("Running on CPU")


Running on CPU


In [70]:
# Data augmentation for the training set
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])


In [71]:
train_dataset_path = "Dataset/Train/"
val_dataset_path = "Dataset/Validation/"
test_dataset_path = "Dataset/Test/" 

# Data loaders
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load datasets
train_dataset = datasets.ImageFolder(train_dataset_path, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, generator=torch.Generator(device))

val_dataset = datasets.ImageFolder(val_dataset_path, transform=train_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8)

test_dataset = datasets.ImageFolder(test_dataset_path, transform=train_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=8)

In [72]:
# Model
model_name = 'efficientnet-b3'
model = EfficientNet.from_pretrained(model_name)

num_epochs = 1
dropout_rate = 0.25  # You can adjust this value

run = wandb.init()

#WandB Sweep
sweep_config = {
    "method": "random",
    "metric": {"name": "loss", "goal": "minimize"},
    "parameters": {
        "num_epochs": {"value": num_epochs},
        "learning_rate": {"values": [0.01, 0.02, 0.03, 0.04]},
        "dropout_rate": {"value": dropout_rate}
}}


# Adding Dropout layer
model._dropout = nn.Dropout(p=dropout_rate)

# Modify the Final Layer for binary classification
num_classes = 2
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, num_classes)

model = model.to(device)

Loaded pretrained weights for efficientnet-b3


In [73]:
# Loss and Optimizer with L2 Regularization (Weight Decay)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) 

In [74]:
sweep_id = wandb.sweep(sweep=sweep_config, project = 'CSC461-Final-Project')

# Training Loop
for epoch in range(num_epochs):  
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Save the model at the end of each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss.item(),
    }, f"model_epoch_{epoch+1}.pth")

wandb.agent(sweep_id, function= model(), count=2)

Create sweep with ID: 149bxlot
Sweep URL: https://wandb.ai/devinhunsberger/CSC461-Final-Project/sweeps/149bxlot
Epoch [1/1], Step [100/4376], Loss: 0.0800
Epoch [1/1], Step [200/4376], Loss: 0.1242


KeyboardInterrupt: 

In [ ]:
best_val_accuracy = 0
# Validation Loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    current_val_accuracy = 100 * correct / total
    print(f"Validation Accuracy: {100 * correct / total}%")
    # Save the model if it has the best validation accuracy so far
    if current_val_accuracy > best_val_accuracy:
        best_val_accuracy = current_val_accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Test Loop with Confusion Matrix
model.eval()
all_labels = []
all_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

conf_mat = confusion_matrix(all_labels, all_preds)
sns.heatmap(conf_mat, annot=True, fmt="d")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Calculate and print test accuracy
correct = sum([1 for i, j in zip(all_labels, all_preds) if i == j])
total = len(all_labels)
print(f"Test Accuracy: {100 * correct / total}%")
